In [1]:
import sys
import math

from matrix import *
from context import *
from nets.gvm import *
from ds.m5 import *
from ds.data_utils import *

np.set_printoptions(suppress=True)

In [2]:
def set_gpu_memory_growth_mode(gpu_id=0):
    import tensorflow as tf
    try:
        gpus = tf.config.experimental.list_physical_devices('GPU')
        tf.config.experimental.set_memory_growth(gpus[gpu_id], True)
    except RuntimeError as e:
        print(e)
set_gpu_memory_growth_mode()

# Context

In [3]:
config = {
    'seed' : 1234,
    'use_64bits': False,
    
    "img_size_x": 96,
    "img_size_y": 64,
    "whole_eye_image_x":96,
    "whole_eye_image_y":48,
    "n_channel": 3,
    
    "batch_size": 48,
    "num_epochs": 100,
    "learning_rate": 0.0005,
    
    "ds_base_path": '/home/elvin/banner/mnt/ssd3/everyone-norm',
    "data_path": '/home/elvin/banner/mnt/ssd3/everyone-norm/valid/',
}

ctx = Context.create(config)

In [4]:
net = M4(ctx)
net.load('sp-norm-61.hdf5')     

Model loaded


In [5]:
train_path = os.path.join(ctx.ds_base_path, "train", "tfrecord4")
valid_path = os.path.join(ctx.ds_base_path, "valid", "tfrecord4")
train_tf_paths = grep_files(train_path, '*.tfrecords')
valid_tf_paths = grep_files(valid_path, '*.tfrecords')

print(len(train_tf_paths))
print(len(valid_tf_paths))

1227
50


In [6]:
def decode(serialized_item):
    record = tf.io.parse_example(
        serialized_item,
        features={
            'subject' : tf.io.FixedLenFeature([], tf.string),
            'frame'   : tf.io.FixedLenFeature([], tf.string),
            'img_re'  : tf.io.VarLenFeature(tf.string),
            'img_le'  : tf.io.VarLenFeature(tf.string),
            'img_we'  : tf.io.VarLenFeature(tf.string),
            'gaze'    : tf.io.FixedLenFeature([6], tf.float32),
            'eyes'    : tf.io.FixedLenFeature([6], tf.float32),
            'poses'   : tf.io.FixedLenFeature([9], tf.float32),
            'rR'      : tf.io.FixedLenFeature([9], tf.float32),
            'lR'      : tf.io.FixedLenFeature([9], tf.float32),
            'cR'      : tf.io.FixedLenFeature([9], tf.float32),
            'gaze2d'  : tf.io.FixedLenFeature([3], tf.float32),
        }
    )
    
    img_re = record['img_re'].values[0]
    img_re = tf.image.decode_jpeg(img_re)[16:-16,:,::-1]
    img_re = tf.image.convert_image_dtype(img_re, tf.float32)

    img_le = record['img_le'].values[0]
    img_le = tf.image.decode_jpeg(img_le)[16:-16,:,::-1]
    img_le = tf.image.convert_image_dtype(img_le, tf.float32)

    img_we = record['img_we'].values[0]
    img_we = tf.image.decode_jpeg(img_we)[:,:,::-1]
    img_we = tf.image.convert_image_dtype(img_we, tf.float32)

    gaze_r  = record['gaze'][0:3]
    gaze_l  = record['gaze'][3:6]
    gaze_cm = record['gaze2d'] / 10

    outputs = (gaze_r, gaze_l, gaze_cm, gaze_cm, gaze_cm)
    return (img_re, img_le, img_we, record['eyes'], record['poses'], 
              record['rR'], record['lR'], record['gaze2d'], record['gaze2d']), outputs


In [7]:
dr = DatasetReader(ctx, phase='tfrecord4', epoch=1)

TypeError: list indices must be integers or slices, not str

In [ ]:
# train_ds = tf.data.TFRecordDataset(train_tf_paths)
# valid_ds = tf.data.TFRecordDataset(valid_tf_paths)
# train_seq = train_ds.map(decode, num_parallel_calls=16).prefetch(tf.data.experimental.AUTOTUNE).batch(16)
# valid_seq = valid_ds.map(decode, num_parallel_calls=16).prefetch(tf.data.experimental.AUTOTUNE).batch(1)

In [ ]:
# net.model.fit(train_seq, shuffle=False, epochs=1)

In [ ]:
res = net.model.predict(valid_seq_s)